In [136]:
# Import the necessary components from Keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [137]:
#i use 100 data to reduce the memory, due to my slow computer ^^'
original_training_path = './cat_dog/training_set'
original_test_set  = './cat_dog/test_set'

# List all files in the original training directory
random_files_training = [os.path.join(dp, f) for dp, dn, filenames in os.walk(original_training_path) for f in filenames if os.path.splitext(f)[1].lower() in ['.png', '.jpg', '.jpeg']]
random_files_training = shuffle(random_files_training)  # Shuffle the list

random_files_test = [os.path.join(dp, f) for dp, dn, filenames in os.walk(original_test_set) for f in filenames if os.path.splitext(f)[1].lower() in ['.png', '.jpg', '.jpeg']]
random_files_test = shuffle(random_files_test)


# Select the first 100 files
random_training_set = random_files_training[:100]
random_test_set = random_files_test[:100]

# Assuming labels are defined by directory names
labels_training_set = [os.path.basename(os.path.dirname(file)) for file in random_training_set]
labels_test_set = [os.path.basename(os.path.dirname(file)) for file in random_test_set]

# Create a DataFrame
df_training_set = pd.DataFrame({'filename': random_training_set, 'class': labels_training_set})
df_test_set = pd.DataFrame({'filename': random_test_set, 'class': labels_test_set})

In [138]:
print(df_training_set)

                                    filename class
0   ./cat_dog/training_set/dogs/dog.3066.jpg  dogs
1   ./cat_dog/training_set/cats/cat.1223.jpg  cats
2   ./cat_dog/training_set/cats/cat.1208.jpg  cats
3    ./cat_dog/training_set/cats/cat.978.jpg  cats
4   ./cat_dog/training_set/cats/cat.1541.jpg  cats
..                                       ...   ...
95  ./cat_dog/training_set/cats/cat.3501.jpg  cats
96  ./cat_dog/training_set/cats/cat.1036.jpg  cats
97  ./cat_dog/training_set/dogs/dog.1002.jpg  dogs
98  ./cat_dog/training_set/cats/cat.2027.jpg  cats
99  ./cat_dog/training_set/cats/cat.3280.jpg  cats

[100 rows x 2 columns]


In [139]:
# Create the ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)

img_height, img_width = 150, 150

train_generator = datagen.flow_from_dataframe(
    dataframe=df_training_set,
    x_col='filename',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale',
    subset='training',  # Use 'training' subset
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=df_test_set,
    x_col='filename',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=10,
    class_mode='binary',
    color_mode='grayscale'
)


Found 100 validated image filenames belonging to 2 classes.
Found 100 validated image filenames belonging to 2 classes.


In [140]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', input_shape=(img_height, img_width, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_43 (Conv2D)          (None, 150, 150, 32)      320       
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 75, 75, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_44 (Conv2D)          (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 37, 37, 64)       0         
 g2D)                                                            
                                                                 
 flatten_23 (Flatten)        (None, 87616)             0         
                                                                 
 dense_30 (Dense)            (None, 1)               

In [141]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [142]:
model.fit(train_generator, validation_data=validation_generator, epochs=10)


Epoch 1/10


2024-05-10 09:19:01.220832: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


4/4 [==============================] - ETA: 0s - loss: 1.2065 - accuracy: 0.4800

2024-05-10 09:19:03.497075: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


4/4 [==============================] - 3s 435ms/step - loss: 1.2065 - accuracy: 0.4800 - val_loss: 0.9272 - val_accuracy: 0.4800
Epoch 2/10
4/4 [==============================] - 1s 340ms/step - loss: 0.7556 - accuracy: 0.5200 - val_loss: 0.6924 - val_accuracy: 0.5300
Epoch 3/10
4/4 [==============================] - 1s 323ms/step - loss: 0.6684 - accuracy: 0.6600 - val_loss: 0.6935 - val_accuracy: 0.5200
Epoch 4/10
4/4 [==============================] - 1s 319ms/step - loss: 0.6499 - accuracy: 0.6700 - val_loss: 0.6954 - val_accuracy: 0.5300
Epoch 5/10
4/4 [==============================] - 1s 345ms/step - loss: 0.6425 - accuracy: 0.5900 - val_loss: 0.6876 - val_accuracy: 0.5200
Epoch 6/10
4/4 [==============================] - 1s 346ms/step - loss: 0.6043 - accuracy: 0.7000 - val_loss: 0.6891 - val_accuracy: 0.5100
Epoch 7/10
4/4 [==============================] - 1s 417ms/step - loss: 0.5674 - accuracy: 0.9000 - val_loss: 0.6910 - val_accuracy: 0.5200
Epoch 8/10
4/4 [===============

In [143]:
model.evaluate(validation_generator)

 3/10 [========>.....................] - ETA: 0s - loss: 0.7221 - accuracy: 0.5000

2024-05-10 09:19:47.087636: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


10/10 [==============================] - 0s 34ms/step - loss: 0.7886 - accuracy: 0.5300


[0.7885962724685669, 0.5299999713897705]

In [145]:
from keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

img_path = './cat_dog/test_set/dogs/dog.4004.jpg'
img = load_img(img_path, target_size=(150, 150), color_mode='grayscale')

# Convert the image to an array
img_array = img_to_array(img)

# Scale the image
img_array /= 255.0

# Expand dimensions to fit the model input
img_array = np.expand_dims(img_array, axis=0)


In [146]:
prediction = model.predict(img_array)
print("Predicted Probability:", prediction)

print(prediction)
# To get a class label, you might threshold the probability at 0.5 (adjust based on your use case)
predicted_class = 'dog' if prediction[0][0] > 0.5 else 'cat'
print("Predicted Class:", predicted_class)

1/1 [==============================] - 0s 177ms/step
Predicted Probability: [[0.8420747]]
Predicted Class: dog
